In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import numpy as np
import pandas as pd
import datetime
import seaborn as sns
import ogb
from tqdm import tqdm
import hiplot as hip
from copy import deepcopy
import datetime

In [3]:
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch.utils.data import Subset, TensorDataset

In [4]:
cwd = os.getcwd()
print(cwd)

/cluster/home/skyriakos/chemprop_run/git/notebooks


In [5]:
os.chdir('..')
import deepadr
from deepadr.dataset import *
from deepadr.utilities import *
from deepadr.run_workflow import *
from deepadr.chemfeatures import *
from deepadr.hyphelperv2 import *
# from deepadr.model_gnn import GCN as testGCN
from deepadr.model_gnn_ogb import GNN, DeepAdr_SiameseTrf, ExpressionNN
# from deepadr.model_attn_siamese import *
from ogb.graphproppred import Evaluator
os.chdir(cwd)

In [6]:
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw

In [7]:
# from tdc.single_pred import Tox
# from tdc.multi_pred import DDI

In [8]:
rawdata_dir = '../data/raw/'
processed_dir = '../data/processed/'
up_dir = '..'

In [9]:
report_available_cuda_devices()

number of GPUs available: 1
cuda:0, name:GeForce GTX 1080 Ti
total memory available: 10.91650390625 GB
total memory allocated on device: 0.0 GB
max memory allocated on device: 0.0 GB
total memory cached on device: 0.0 GB
max memory cached  on device: 0.0 GB



In [10]:
n_gpu = torch.cuda.device_count()
n_gpu

1

In [11]:
device_cpu = get_device(to_gpu=False)
# device_gpu = get_device(True, index=0)

# fdtype = torch.float32

In [12]:
print("torch:", torch.__version__)
print("CUDA:", torch.version.cuda)
print(sys.version)

torch: 1.9.1
CUDA: 11.1
3.9.9 | packaged by conda-forge | (main, Dec 20 2021, 02:41:03) 
[GCC 9.4.0]


In [13]:
# print(torch.cuda.memory_summary(device=device_gpu))

### Preparing dataset 

In [14]:
score = 'total_thresh'
score_val = 4

In [15]:
# TDC Tox
# DSdataset_name = 'SynergxZloewe' #'OncoPolyPharmacology' #'DrugComb'
DSdataset_name = f'DrugComb_{score}_{score_val}' #'DrugComb'


#fname_suffix = ds_config["fname_suffix"]
# similarity_types = ['chem']
# kernel_option = 'sqeuclidean'
data_fname = 'data_v1'
# interact_matfname = ds_config["interact_matfname"]
# exp_iden = 'simtypeall'
# ddi_interaction_labels_pth = ds_config["ddi_interaction_labels_pth"]

# up_dir, processed_dir, DSdataset_name, data_fname

In [16]:
targetdata_dir = create_directory(os.path.join(processed_dir, DSdataset_name, data_fname))
targetdata_dir_raw = create_directory(os.path.join(targetdata_dir, "raw"))
targetdata_dir_processed = create_directory(os.path.join(targetdata_dir, "processed"))
targetdata_dir_exp = create_directory(os.path.join(targetdata_dir, "experiments"))
# # ReaderWriter.dump_data(dpartitions, os.path.join(targetdata_dir, 'data_partitions.pkl'))
print(targetdata_dir)

path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
/cluster/home/skyriakos/chemprop_run/git/data/processed/DrugComb_total_thresh_4/data_v1


In [17]:
%%time

# Make sure to first run the "data_generation" notebook first

dataset = MoleculeDataset(root=targetdata_dir)

CPU times: user 12.2 ms, sys: 125 ms, total: 137 ms
Wall time: 134 ms


In [18]:
# print()
# print(f'Dataset: {dataset}:')
# print('====================')
# print(f'Number of graphs: {len(dataset)}')
# print(f'Number of features: {dataset.num_features}')
# print(f'Number of classes: {dataset.num_classes}')

# # data0 = dataset[0]  # Get the first graph object.

# # print()
# # print(data)
# # print('=============================================================')

# # # Gather some statistics about the first graph.
# # print(f'Number of nodes: {data.num_nodes}')
# # print(f'Number of edges: {data.num_edges}')
# # print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
# # print(f'Contains isolated nodes: {data.contains_isolated_nodes()}')
# # print(f'Contains self-loops: {data.contains_self_loops()}')
# # print(f'Is undirected: {data.is_undirected()}')

In [19]:
# data0

In [20]:
# data0.expression.shape

In [21]:
# data0.y

In [22]:
# dataset.tensors[1]

In [23]:
# used_dataset = dataset

# If you want to use a smaller subset of the dataset for testing
# smaller_dataset_len = int(len(dataset)/1)
# used_dataset = dataset[:smaller_dataset_len]

In [24]:
# type(used_dataset)

In [25]:
dataset.data.y

tensor([0, 0, 0,  ..., 0, 1, 1], dtype=torch.int32)

In [26]:
fold_partitions = get_stratified_partitions(dataset.data.y,
                                            num_folds=5,
                                            valid_set_portion=0.1,
                                            random_state=42)

fold_num: 0
train data
class: 0 norm count: 0.5764128559102675
class: 1 norm count: 0.4235871440897325
validation data
class: 0 norm count: 0.5764192139737991
class: 1 norm count: 0.42358078602620086
test data
class: 0 norm count: 0.5764751552795031
class: 1 norm count: 0.4235248447204969

-------------------------
fold_num: 1
train data
class: 0 norm count: 0.5764128559102675
class: 1 norm count: 0.4235871440897325
validation data
class: 0 norm count: 0.5764192139737991
class: 1 norm count: 0.42358078602620086
test data
class: 0 norm count: 0.5764751552795031
class: 1 norm count: 0.4235248447204969

-------------------------
fold_num: 2
train data
class: 0 norm count: 0.5764356969533567
class: 1 norm count: 0.4235643030466433
validation data
class: 0 norm count: 0.5764192139737991
class: 1 norm count: 0.42358078602620086
test data
class: 0 norm count: 0.5763929334109882
class: 1 norm count: 0.4236070665890118

-------------------------
fold_num: 3
train data
class: 0 norm count: 0.576

In [27]:
# train_val_test_frac = [0.7, 0.1, 0.2]
# assert sum(train_val_test_frac) == 1

# torch.manual_seed(42)
# used_dataset = used_dataset.shuffle()

# num_train = round(train_val_test_frac[0] * len(used_dataset)) 
# num_trainval = round((train_val_test_frac[0]+train_val_test_frac[1]) * len(used_dataset))

# train_dataset = used_dataset[:num_train]
# val_dataset = used_dataset[num_train:num_trainval]
# test_dataset = used_dataset[num_trainval:]

In [28]:
# train_dataset = Subset(used_dataset, fold_partitions[0]['train'])
# val_dataset = Subset(used_dataset, fold_partitions[0]['validation'])
# test_dataset = Subset(used_dataset, fold_partitions[0]['test'])

In [29]:
# print(f'Number of training graphs: {len(train_dataset)}')
# print(f'Number of val graphs: {len(val_dataset)}')
# print(f'Number of test graphs: {len(test_dataset)}')

In [30]:
print("Number of training graphs: "+ str(len(fold_partitions[0]['train'])))
print("Number of validation graphs: "+ str(len(fold_partitions[0]['validation'])))
print("Number of testing graphs: "+ str(len(fold_partitions[0]['test'])))


Number of training graphs: 18544
Number of validation graphs: 2061
Number of testing graphs: 5152


In [31]:
# print(dataset.num_classes)

In [32]:
# # training params total_thresh 4
# tp = {
#     "batch_size" : 300,
#     "num_epochs" : 30,
    
#     "emb_dim" : 300,
#     "gnn_type" : "gatv2",
#     "num_layer" : 5,
#     "graph_pooling" : "mean", #attention
    
#     "input_embed_dim" : None,
#     "gene_embed_dim": 1,
#     "num_attn_heads" : 2,
#     "num_transformer_units" : 1,
#     "p_dropout" : 0.3,
# #     "nonlin_func" : nn.ReLU(),
#     "mlp_embed_factor" : 2,
#     "pooling_mode" : 'attn',
#     "dist_opt" : 'cosine',

#     "base_lr" : 3e-5, #3e-4
#     "max_lr_mul": 15,
#     "l2_reg" : 1e-7,
#     "loss_w" : 0.95,
#     "margin_v" : 1.,

#     "expression_dim" : 64,
#     "expression_input_size" : 908,
#     "exp_H1" : 500,
#     "exp_H2" : 400
# }

In [33]:
tp = {
    "batch_size" : 300,
    "num_epochs" : 30,
    
    "emb_dim" : 300,
    "gnn_type" : "gatv2",
    "num_layer" : 5,
    "graph_pooling" : "mean", #attention
    
    "input_embed_dim" : None,
    "gene_embed_dim": 1,
    "num_attn_heads" : 2,
    "num_transformer_units" : 1,
    "p_dropout" : 0.3,
#     "nonlin_func" : nn.ReLU(),
    "mlp_embed_factor" : 2,
    "pooling_mode" : 'attn',
    "dist_opt" : 'cosine',

    "base_lr" : 3e-5, #3e-4
    "max_lr_mul": 5,
    "l2_reg" : 1e-5,
    "loss_w" : 1.,
    "margin_v" : 1.,

    "expression_dim" : 64,
    "expression_input_size" : 908,
    "exp_H1" : 500,
    "exp_H2" : 400
}

In [34]:
# # training params total_thresh 3
# tp = {
#     "batch_size" : 300,
#     "num_epochs" : 200,
    
#     "emb_dim" : 300,
#     "gnn_type" : "gatv2",
#     "num_layer" : 5,
#     "graph_pooling" : "mean", #attention
    
#     "input_embed_dim" : None,
#     "gene_embed_dim": 1,
#     "num_attn_heads" : 2,
#     "num_transformer_units" : 1,
#     "p_dropout" : 0.3,
# #     "nonlin_func" : nn.ReLU(),
#     "mlp_embed_factor" : 2,
#     "pooling_mode" : 'attn',
#     "dist_opt" : 'cosine',

#     "base_lr" : 3e-5, #3e-4
#     "max_lr_mul": 10,
#     "l2_reg" : 1e-8,
#     "loss_w" : 0.3,
#     "margin_v" : 1.,

#     "expression_dim" : 64,
#     "expression_input_size" : 908,
#     "exp_H1" : 500,
#     "exp_H2" : 400
# }

In [35]:
# targetdata_dir_exp = create_directory(os.path.join(targetdata_dir, "experiments"))
# targetdata_dir_exp

In [36]:
# device_gpu = get_device(True, index=0)
# exp_dir = create_directory(os.path.join(targetdata_dir_exp, "exp0"))
# tphp = generate_tp_hp(tp, hyperparam_space[0], hp_names)
# partition = fold_partitions[0]

In [37]:
# run_exp(tphp, device_gpu, exp_dir,partition=fold_partitions[0], used_dataset=used_dataset)

In [38]:
def spawn_q_process(q_process):
    print(">>> spawning hyperparam search process")
    q_process.start()
    
def join_q_process(q_process):
    q_process.join()
    print("<<< joined hyperparam search process")
    
def create_q_process(queue, used_dataset, gpu_num, tphp, exp_dir, partition): #
#     fold_gpu_map = {0:gpu_num}
    return mp.Process(target=deepadr.hyphelperv2.run_exp, args=(queue, used_dataset, gpu_num, tphp, exp_dir, partition)) #

In [39]:
import torch.multiprocessing as mp
mp.set_start_method("spawn", force=True)

queue = mp.Queue()
q_processes = []

# partition = fold_partitions[0]
time_stamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

print("Start: " + time_stamp)

for q_i in range(min(n_gpu, len(fold_partitions))):
#     device_gpu = get_device(True, index=q_i)
    partition = fold_partitions[q_i]
    exp_dir = create_directory(os.path.join(targetdata_dir_exp, "fold_"+str(q_i)+"_"+time_stamp))
    create_directory(os.path.join(exp_dir, "predictions"))

#     tphp = generate_tp_hp(tp, hyperparam_space[q_i], hp_names)
    
    q_process = create_q_process(queue, dataset, q_i, tp, exp_dir, partition)
    q_processes.append(q_process)
    spawn_q_process(q_process)

spawned_processes = n_gpu
    
# for q_i in range(len(hyperparam_space)):
for q_i in range(min(n_gpu, len(fold_partitions))):
    join_q_process(q_processes[q_i])
    released_gpu_num = queue.get()
    print("released_gpu_num:", released_gpu_num)
#     if(spawned_processes < len(hyperparam_space)):
# #         device_gpu = get_device(True, index=q_i)
#         time_stamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
#         exp_dir = create_directory(os.path.join(targetdata_dir_exp, "exp_"+str(q_i)+"_"+time_stamp))
#         tphp = generate_tp_hp(tp, hyperparam_space[q_i], hp_names)

#         q_process = create_q_process(queue, used_dataset, released_gpu_num, tphp, exp_dir, partition)
#         q_processes.append(q_process)
#         spawn_q_process(q_process)
#         spawned_processes = spawned_processes + 1

Start: 2022-03-24_12-43-52
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
>>> spawning hyperparam search process
gpu: cuda:0
=====Epoch 0
Training...


Iteration:   0%|          | 0/62 [00:00<?, ?it/s]

Evaluating...


Iteration: 100%|██████████| 62/62 [00:04<00:00, 15.15it/s]
/opt/conda/envs/graphnn/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/graphnn/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/graphnn/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mod

{'Train':  best_epoch_indx:0
 auc:0.6524802590978704 
 apur:0.5711178684951095 
 f1:0.0 
 precision:0.0 
 recall:0.0 
, 'Validation':  best_epoch_indx:0
 auc:0.6498866095086026 
 apur:0.5613938901407765 
 f1:0.0 
 precision:0.0 
 recall:0.0 
, 'Test':  best_epoch_indx:0
 auc:0.6520861687451971 
 apur:0.5635944170836358 
 f1:0.0 
 precision:0.0 
 recall:0.0 
}
=====Epoch 1
Training...


...

{'Train':  best_epoch_indx:28
 auc:0.9075506750992814 
 apur:0.8844405337136625 
 f1:0.7964217252396165 
 precision:0.7994868505452213 
 recall:0.7933800127307448 
, 'Validation':  best_epoch_indx:28
 auc:0.8633837419633525 
 apur:0.8326099157187824 
 f1:0.7478260869565218 
 precision:0.7570422535211268 
 recall:0.738831615120275 
, 'Test':  best_epoch_indx:28
 auc:0.8771432627527954 
 apur:0.848201749162091 
 f1:0.7644197359277276 
 precision:0.7728337236533958 
 recall:0.7561869844179652 
}
=====Epoch 29
Training...


Iteration:   0%|          | 0/62 [00:00<?, ?it/s]

Evaluating...


Iteration: 100%|██████████| 7/7 [00:00<00:00, 15.84it/s]


{'Train':  best_epoch_indx:29
 auc:0.9108413862231524 
 apur:0.8887167135323067 
 f1:0.8016523830075836 
 precision:0.7772596843615495 
 recall:0.8276257161043921 
, 'Validation':  best_epoch_indx:29
 auc:0.8635418715601992 
 apur:0.8332910570816917 
 f1:0.7536557930258718 
 precision:0.7403314917127072 
 recall:0.7674684994272624 
, 'Test':  best_epoch_indx:29
 auc:0.8776158313967664 
 apur:0.8493170054888168 
 f1:0.7653606411398041 
 precision:0.7441558441558441 
 recall:0.7878093492208983 
}
Finished training!
<<< joined hyperparam search process
released_gpu_num: 0


In [40]:
print("End: " + datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))


End: 2022-03-24_12-53-52
